In [1]:
import torch
import torch_geometric
from torch_geometric import utils
from torch_geometric.nn import SAGEConv
import torch.nn as nn
import torch.nn.functional as F

from scipy.sparse import coo_matrix
import pandas as pd
import numpy as np

In [ ]:
# don't run - cell containing code for combining graph to two subparts with offset
# G_ec = nx.read_graphml("ec_graph.graphml")
# G_hsr = nx.read_graphml("hsr_graph.graphml")
# ec_graph = from_networkx(G_ec)
# hsr_graph = from_networkx(G_hsr)

# # Shift node indices of ec Graph to ensure disjointness
# offset = ec_graph.x.size(0)  # Number of nodes in ec graph 
# edge_index_ec = edge_index_ec + offset

# # Combine edge indices and node features
# combined_edge_index = torch.cat([hsr_graph.edge_index, edge_index_ec], dim=1)
# combined_x = torch.cat([hsr_graph.x, ec_graph.x], dim=0)

# # Create a new Data object for the combined graph
# combined_graph = Data(edge_index=combined_edge_index, x=combined_x)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
ec_hic = np.load('data/GBM39ec_5k_collapsed_matrix.npy')
hsr_hic = np.load('data/GBM39HSR_5k_collapsed_matrix.npy')

In [3]:
ec_df = pd.read_csv('data/ec_cleaned.csv')
hsr_df = pd.read_csv('data/hsr_cleaned.csv')

In [4]:
hsr_feats = torch.tensor(hsr_df[['read_count', 'total_genes']].to_numpy())
hsr_labels = torch.zeros(hsr_feats.shape[0])

ec_feats = torch.tensor(ec_df[['read_count', 'total_genes']].to_numpy())
ec_labels = torch.ones(ec_feats.shape[0])

In [5]:
def hic_to_sparse(hic_mat):
    adj_mat = np.triu(hic_mat)
    sparse_adj = coo_matrix(adj_mat)

    return utils.from_scipy_sparse_matrix(sparse_adj)

In [6]:
hsr_edge_index, hsr_edge_attr = hic_to_sparse(hsr_hic)
hsr_graph = torch_geometric.data.Data(edge_index = hsr_edge_index, edge_attr = hsr_edge_attr, x = hsr_feats, y = hsr_labels)

ec_edge_index, ec_edge_attr = hic_to_sparse(ec_hic)
ec_graph = torch_geometric.data.Data(edge_index = ec_edge_index, edge_attr = ec_edge_attr, x = ec_feats, y = ec_labels)

In [7]:
x = torch.cat([ec_feats, hsr_feats], dim=0)
hsr_edge_index = hsr_edge_index + ec_labels.shape[0]
edge_index = torch.cat([ec_edge_index, hsr_edge_index], dim=1)
edge_attr = torch.cat([ec_edge_attr, hsr_edge_attr], dim=0)
labels = torch.cat([ec_labels, hsr_labels], dim=0)

G = torch_geometric.data.Data(edge_index = edge_index, edge_attr = edge_attr, x = x, y = labels)

In [8]:
class GraphSAGE(nn.Module):
    def __init__(self, num_feat, num_graph_conv_layers, graph_conv_layer_sizes, num_lin_layers, lin_hidden_sizes, num_classes):
        super().__init__()
        self.num_graph_conv_layers = num_graph_conv_layers
        self.num_lin_layers = num_lin_layers
        self.embeds = None

        if self.num_graph_conv_layers == 1:
            self.conv1 = SAGEConv(graph_conv_layer_sizes[0], graph_conv_layer_sizes[1])
        elif self.num_graph_conv_layers == 2:
            self.conv1 = SAGEConv(graph_conv_layer_sizes[0], graph_conv_layer_sizes[1])
            self.conv2 = SAGEConv(graph_conv_layer_sizes[1], graph_conv_layer_sizes[2])
        elif self.num_graph_conv_layers == 3:
            self.conv1 = SAGEConv(graph_conv_layer_sizes[0], graph_conv_layer_sizes[1])
            self.conv2 = SAGEConv(graph_conv_layer_sizes[1], graph_conv_layer_sizes[2])
            self.conv3 = SAGEConv(graph_conv_layer_sizes[2], graph_conv_layer_sizes[3])
        
        if self.num_lin_layers == 1:
            self.lin1 = nn.Linear(lin_hidden_sizes[0], lin_hidden_sizes[1])
        elif self.num_lin_layers == 2:
            self.lin1 = nn.Linear(lin_hidden_sizes[0], lin_hidden_sizes[1])
            self.lin2 = nn.Linear(lin_hidden_sizes[1], lin_hidden_sizes[2])
        elif self.num_lin_layers == 3:
            self.lin1 = nn.Linear(lin_hidden_sizes[0], lin_hidden_sizes[1])
            self.lin2 = nn.Linear(lin_hidden_sizes[1], lin_hidden_sizes[2])
            self.lin3 = nn.Linear(lin_hidden_sizes[2], lin_hidden_sizes[3])
            
        self.loss_calc = nn.CrossEntropyLoss()
        self.torch_softmax = nn.Softmax(dim=1)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr

        ### Graph convolution module
        if self.num_graph_conv_layers == 1:
            h = self.conv1(x, edge_index)
            h = torch.relu(h)
        elif self.num_graph_conv_layers == 2:
            h = self.conv1(x, edge_index)
            h = torch.relu(h)
            h = self.conv2(h, edge_index)
            h = torch.relu(h)
        elif self.num_graph_conv_layers == 3:
            h = self.conv1(x, edge_index)
            h = torch.relu(h)
            h = self.conv2(h, edge_index)
            h = torch.relu(h)
            h = self.conv3(h, edge_index)
            h = torch.relu(h)
            
        #h = F.dropout(h, p = self.dropout_value)
        scores = h
        ### Linear module
        if self.num_lin_layers == 0:
            return scores
        elif self.num_lin_layers == 1:
            scores = self.lin1(scores)
        elif self.num_lin_layers == 2:
            scores = self.lin1(scores)
            scores = torch.relu(scores)
            scores = self.lin2(scores)
        elif self.num_lin_layers == 3:
            scores = self.lin1(scores)
            scores = torch.relu(scores)
            scores = self.lin2(scores)
            scores = torch.relu(scores)
            scores = self.lin3(scores)
        
        return scores

    def loss(self, scores, labels):
        xent_loss = self.loss_calc(scores, labels)
        return xent_loss
    
    
    def calc_softmax_pred(self, scores):
        softmax = self.torch_softmax(scores)
        predicted = torch.argmax(softmax, 1)
        return softmax, predicted


In [10]:
# separate train/test sets
def split_data(G, train_ratio=0.8, test_ratio=0.2, seed=42):
    assert train_ratio + test_ratio == 1, "Ratios must sum to 1."
    
    torch.manual_seed(seed)  # For reproducibility
    
    num_nodes = G.x.shape[0]  # Total number of nodes
    indices = torch.randperm(num_nodes)  # Shuffle node indices randomly
    
    # Compute split size
    train_size = int(train_ratio * num_nodes)
    
    # Assign indices to train and test sets
    train_idx = indices[:train_size]
    test_idx = indices[train_size:]

    # Create boolean masks
    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    train_mask[train_idx] = True
    test_mask[test_idx] = True

    # Attach masks to the graph
    G.train_mask = train_mask
    G.test_mask = test_mask

    return G

In [15]:
# build train function with train/test split, build test function - condensed
def train(model, optimizer, graph, device):
    model.train()
    optimizer.zero_grad()
    scores = model(graph)
    loss = model.loss(scores[graph.train_mask], graph.y[graph.train_mask])
    loss.backward()
    optimizer.step()
    
    return loss.item(), scores

def test(model, graph, device):
    model.eval()
    scores = model(graph)
    softmax, predicted = model.calc_softmax_pred(scores)
    accs = []
    for mask in [graph.train_mask, graph.test_mask]:
        correct = (predicted[mask] == graph.y[mask]).sum().item()
        acc = correct / mask.sum().item()
        accs.append(acc)
    
    return tuple(accs)

In [16]:
# Model inputs + layer information
num_features = G.num_node_features
num_graph_sage_layers = 3
num_classes = 2
graph_sage_layer_sizes = [num_features,8,16,32]
linear_layer_sizes = [32,16,8,num_classes]
num_linear_layers = 3

# Hyperparameters
learning_rate = 0.01
weight_decay = 5e-4
num_epochs = 1000
#dropout_value = 0.5

# Initialize the model, optimizer, and send to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GraphSAGE(num_features, num_graph_sage_layers, graph_sage_layer_sizes, linear_layer_sizes, num_linear_layers, num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
#model.dropout_value = dropout_value

# Assign train and test masks to graph
G.x = G.x.float()
G.y = G.y.long()
G = split_data(G)
G = G.to(device)

In [18]:
train_accs = []
test_accs = []

# Training loop - Entire graph in one model
for epoch in range(1, num_epochs + 1):
    loss, _ = train(model, optimizer, G, device)
    train_acc, test_acc = test(model, G, device)
    train_accs.append(train_acc)
    test_accs.append(test_acc)
    
    if epoch % 100 == 0 or epoch == num_epochs:
        print(f"Epoch {epoch:03d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}")

Epoch 100, Loss: 2.8576, Train Acc: 0.6883, Test Acc: 0.6931
Epoch 200, Loss: 2.0698, Train Acc: 0.7307, Test Acc: 0.7327
Epoch 300, Loss: 0.0295, Train Acc: 1.0000, Test Acc: 0.9802
Epoch 400, Loss: 0.0182, Train Acc: 1.0000, Test Acc: 0.9802
Epoch 500, Loss: 0.0134, Train Acc: 1.0000, Test Acc: 0.9802
Epoch 600, Loss: 0.0107, Train Acc: 1.0000, Test Acc: 0.9802
Epoch 700, Loss: 0.0088, Train Acc: 1.0000, Test Acc: 0.9802
Epoch 800, Loss: 0.0075, Train Acc: 1.0000, Test Acc: 0.9802
Epoch 900, Loss: 0.0065, Train Acc: 1.0000, Test Acc: 0.9802
Epoch 1000, Loss: 0.0058, Train Acc: 1.0000, Test Acc: 0.9802


With 2 hidden layers: sizes 32 and 64:
- First ran with 200 epochs - loss bounced around a bit but didn't converge
- Then ran with 2000 epochs - loss leveled out a bit around 1000, ended with about 88% train acc and 86% test acc. Similar to above with 5000 epochs but there is still obviously inconsistency in the training
- 5000 epoch train - it eventually learns an even split and can achieve 100% accuracy. Going to try learning embeddings from separate models for each graph and classifying using lightgbm or xgboost

With 3 hidden layers sizes 8, 16, and 32:
- training is much more stable; loss decreases gradually
- accuracy still quickly reaches 97+% and stays there. Maybe it found a set of parameters that very accurately differentiates ecDNA and HSR, or it is just figuring out which subgraph the test nodes come from

In [63]:
num_linear_layers = 0
linear_layer_sizes = []

# Hyperparameters
learning_rate = 0.01
weight_decay = 5e-4
num_epochs = 1000
#dropout_value = 0.5

# Initialize the model, optimizer, and send to device
model_ec = GraphSAGE(num_features, num_graph_sage_layers, graph_sage_layer_sizes, linear_layer_sizes, num_linear_layers, num_classes).to(device)
optimizer_ec = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
model_hsr = GraphSAGE(num_features, num_graph_sage_layers, graph_sage_layer_sizes, linear_layer_sizes, num_linear_layers, num_classes).to(device)
optimizer_hsr = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
#model.dropout_value = dropout_value

ec_graph.x = ec_graph.x.float()
ec_labels = ec_graph.y
ec_graph = ec_graph.to(device)

hsr_graph.x = hsr_graph.x.float()
hsr_labels = hsr_graph.y
hsr_graph = hsr_graph.to(device)

In [64]:
# Training loop
def learn_embeddings(model, optimizer, graph, device):
    scores = 0
    for epoch in range(1, num_epochs + 1):
        loss, scores = train(model, optimizer, G, device)
        
        if epoch % 100 == 0 or epoch == num_epochs:
            print(f"Epoch {epoch:03d}, Loss: {loss:.4f}")
    return scores

In [65]:
ec_embeddings = learn_embeddings(model_ec, optimizer_ec, ec_graph, device)
hsr_embeddings = learn_embeddings(model_hsr, optimizer_hsr, hsr_graph, device)

Epoch 100, Loss: 704.1915
Epoch 200, Loss: 704.1915
Epoch 300, Loss: 704.1915
Epoch 400, Loss: 704.1915
Epoch 500, Loss: 704.1915
Epoch 600, Loss: 704.1915
Epoch 700, Loss: 704.1915
Epoch 800, Loss: 704.1915
Epoch 900, Loss: 704.1915
Epoch 1000, Loss: 704.1915
Epoch 100, Loss: 1041.8651
Epoch 200, Loss: 1041.8651
Epoch 300, Loss: 1041.8651
Epoch 400, Loss: 1041.8651
Epoch 500, Loss: 1041.8651
Epoch 600, Loss: 1041.8651
Epoch 700, Loss: 1041.8651
Epoch 800, Loss: 1041.8651
Epoch 900, Loss: 1041.8651
Epoch 1000, Loss: 1041.8651


In [ ]:
# Not learning anything - confused on how to define loss so it can learn embeddings